### 보스턴 주택가격 데이터를 Spark DataFrame으로 로드

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

# 사이킷런 보스턴 주택가격 데이터 세트를 로드
boston = load_boston()
boston_data = boston.data
boston_label = boston.target
boston_columns = boston.feature_names

# boston 데이터 세트를 numpy에서 pandas DataFrame으로 변환 
boston_pdf = pd.DataFrame(boston_data, columns=boston_columns)
boston_pdf['price'] = boston_label

# pandas DataFrame을 Spark DataFrame으로 변환
boston_sdf = spark.createDataFrame(boston_pdf)
display(boston_sdf.limit(5))

* CRIM: 지역별 범죄 발생률  
* ZN: 25,000평방피트를 초과하는 거주 지역의 비율
* INDUS: 비상업 지역 넓이 비율
* CHAS: 찰스강에 대한 더미 변수(강의 경계에 위치한 경우는 1, 아니면 0)
* NOX: 일산화질소 농도
* RM: 거주할 수 있는 방 개수
* AGE: 1940년 이전에 건축된 소유 주택의 비율
* DIS: 5개 주요 고용센터까지의 가중 거리
* RAD: 고속도로 접근 용이도
* TAX: 10,000달러당 재산세율
* PTRATIO: 지역의 교사와 학생 수 비율
* B: 지역의 흑인 거주 비율
* LSTAT: 하위 계층의 비율
* MEDV: 본인 소유의 주택 가격(중앙값)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# 2개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x2개의 ax를 가짐.
fig, axs = plt.subplots(figsize=(16,8) , ncols=4 , nrows=2)
lm_features = ['RM','ZN','INDUS','NOX','AGE','PTRATIO','LSTAT','RAD']
colors = ['g', 'r', 'b', 'c', 'm', 'y', 'orange', 'darkblue' ]
for i , feature in enumerate(lm_features):
    row = int(i/4)
    col = i%4
    # 시본의 regplot을 이용해 산점도와 선형 회귀 직선을 함께 표현
    sns.regplot(x=feature , y='price',data=boston_pdf , ax=axs[row][col], color=colors[i])

### Feature Vectorization 적용하고 학습과 테스트 데이터 세트로 분할

In [0]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols=boston_columns, outputCol='features')
boston_sdf_vectorized = vector_assembler.transform(boston_sdf)
train_sdf, test_sdf = boston_sdf_vectorized.randomSplit([0.7, 0.3], seed=2021)

display(train_sdf.limit(10))

### LinearRegression 학습, 예측, 평가 수행. 
* regParam = 0, elasticNetParam = 0 => 무규제 선형회귀
* regParam > 0, elasticNetParam = 0 => Ridge(L2 규제)
* regParam > 0, elasticNetParam = 1 => Lasso(L1 규제)
* regParam > 0, elasticNetParam = (0 ~ 1) => ElasticNet
* LinearRegression은 데이터를 학습전에 Standard Scaling을 적용할 수 있음. standardization=True가 기본임.

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='price', 
                         maxIter=100, regParam=0)
lr_model = lr.fit(train_sdf)
lr_predictions = lr_model.transform(test_sdf)
display(lr_predictions.limit(10))

In [0]:
# Ridge, Lasso, ElasticNet 학습,예측 테스트를 위해서 함수 생성. 
def do_train_predict(lr_estimator, train_sdf, test_sdf):
    lr_model = lr_estimator.fit(train_sdf)
    predictions = lr_model.transform(test_sdf)
    return lr_model, predictions

lr_model, lr_predictions = do_train_predict(lr, train_sdf, test_sdf)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

def get_reg_eval(predictions):
    mse_eval = RegressionEvaluator(labelCol='price', predictionCol='prediction', metricName='mse')
    rmse_eval = RegressionEvaluator(labelCol='price', predictionCol='prediction', metricName='rmse')
    r2_eval = RegressionEvaluator(labelCol='price', predictionCol='prediction', metricName='r2')

    print('mse:', mse_eval.evaluate(predictions), 'rmse:', rmse_eval.evaluate(predictions), 'r2:', r2_eval.evaluate(predictions))
    
get_reg_eval(lr_predictions)

In [0]:
# LinearRegression은 학습 데이터를 기본으로 정규화 scaling변환. standardization=True가 기본값. 
lr_model.extractParamMap()

In [0]:
# 학습 데이터를 학습전에 정규환 변환 적용하지 않음. 
lr = LinearRegression(featuresCol='features', labelCol='price', 
                         maxIter=100, regParam=0, standardization=False)
lr_model, lr_predictions = do_train_predict(lr, train_sdf, test_sdf)
get_reg_eval(lr_predictions)

#### 회귀 계수(coefficients)와 절편(intercept) 확인

In [0]:
# linear regression의 회귀계수와 절편은 각각 EstimatorModel의 coefficients와 intercept에서 확인 
print('회귀 계수:', lr_model.coefficients)
print('회귀 절편:', lr_model.intercept)

In [0]:
coeff = pd.Series(data=lr_model.coefficients, index=boston_columns)
print(coeff.sort_values(ascending=False))

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

def get_coefficient(coefficients, columns):
    coeff = pd.Series(data=coefficients, index=columns).sort_values(ascending=False)
    print(coeff)
    sns.barplot(x=coeff.values, y=coeff.index)
    plt.show()

get_coefficient(lr_model.coefficients, boston_columns)

### 규제 선형회귀 적용
* regParam = 0, elasticNetParam = 0 => 무규제 선형회귀
* regParam > 0, elasticNetParam = 0 => Ridge(L2 규제)
* regParam > 0, elasticNetParam = 1 => Lasso(L1 규제)
* regParam > 0, elasticNetParam = (0 ~ 1) => ElasticNet

In [0]:
def do_train_predict(lr_estimator, train_sdf, test_sdf):
    lr_model = lr_estimator.fit(train_sdf)
    predictions = lr_model.transform(test_sdf)
    return lr_model, predictions

In [0]:
from pyspark.ml.regression import LinearRegression

# regParam=5, elasticNetParam=0으로 alpha값이 5인 Ridge Estimator 생성. 
ridge = LinearRegression(featuresCol='features', labelCol='price', 
                         maxIter=100, regParam=5, elasticNetParam=0)

ridge_model, ridge_predictions = do_train_predict(ridge, train_sdf, test_sdf)
get_reg_eval(ridge_predictions)
get_coefficient(ridge_model.coefficients, boston_columns)
# mse: 21.564913322698093 rmse: 4.643803755834014 r2: 0.7383398763030963
# mse: 23.721435164891943 rmse: 4.8704656004217854 r2: 0.7121734937380622

In [0]:
from pyspark.ml.regression import LinearRegression

# regParam=0.01, elasticNetParam=1으로 alpha값이 0.1인 Lasso Estimator 생성. 
lasso = LinearRegression(featuresCol='features', labelCol='price', 
                         maxIter=100, regParam=0.1, elasticNetParam=1)

lasso_model, lasso_predictions = do_train_predict(lasso, train_sdf, test_sdf)
get_reg_eval(lasso_predictions)
get_coefficient(lasso_model.coefficients, boston_columns)

In [0]:
from pyspark.ml.regression import LinearRegression

# regParam=20, elasticNetParam=0.1으로 a+b=20, L1 ratio=0.1임. a/(a+b) = 2/20, L1 alpha값이 2, L2 alpha값이 18인 ElasticNet Estimator 생성. 
elastic_net = LinearRegression(featuresCol='features', labelCol='price', 
                         maxIter=100, regParam=20, elasticNetParam=0.1)

elastic_net_model, elastic_net_predictions = do_train_predict(elastic_net, train_sdf, test_sdf)
get_reg_eval(elastic_net_predictions)
get_coefficient(elastic_net_model.coefficients, boston_columns)

### 선형 회귀 Scaling 적용

In [0]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

# 전체 컬럼에 Standard Scaler 적용. scaling은 vectorized된 feature에만 가능. feature vectorization 적용후 standard scaling 적용. 
vec_assembler = VectorAssembler(inputCols=boston_columns, outputCol='features')
standard_scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

boston_sdf_vectorized = vec_assembler.transform(boston_sdf)
boston_sdf_vect_scaled = standard_scaler.fit(boston_sdf_vectorized).transform(boston_sdf_vectorized)

display(boston_sdf_vect_scaled.limit(10))

In [0]:
# feature vectorization->scaling이 적용된 데이터 세트를 학습과 테스트로 분리
train_sdf_scaled, test_sdf_scaled = boston_sdf_vect_scaled.randomSplit([0.7, 0.3], seed=2021)

#featuresCol이 features가 아닌 scaled_features가 되어야함. 
ridge_scale = LinearRegression(featuresCol='scaled_features', labelCol='price', 
                         maxIter=100, regParam=5, elasticNetParam=0, standardization=False)

# scaled된 학습과 테스트 데이터 세트 입력하여 lasso 학습/예측/평가
ridge_model, ridge_predictions = do_train_predict(ridge_scale, train_sdf_scaled, test_sdf_scaled)
get_reg_eval(ridge_predictions)
get_coefficient(ridge_model.coefficients, boston_columns)

#mse: 21.564913322698093 rmse: 4.643803755834014 r2: 0.7383398763030963

### 회귀 트리 적용
* DecisionTreeRegressor, RandomForestRegressor, GBTRegressor 에서 RandomForestRegressor만 적용해봄.

In [0]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol='features', labelCol='price', 
                               maxDepth=5, numTrees=10)
rf_model, rf_predictions = do_train_predict(rf, train_sdf, test_sdf)
get_reg_eval(rf_predictions)

In [0]:
from pyspark.ml.linalg import DenseVector
import matplotlib.pyplot as plt
import seaborn as sns

# feature importance가 Sparse Vector이므로 Dense Vector로 변환. 
rf_ftr_importances_list = list(DenseVector(rf_model.featureImportances))

ftr_importances = pd.Series(data=rf_ftr_importances_list, index=boston_columns).sort_values(ascending=False)
print(ftr_importances)
sns.barplot(x=ftr_importances.values, y=ftr_importances.index)
plt.show()